In [21]:
import pandas as pd
import numpy as np
from math import cos, asin, sqrt


In [146]:
vuelos = pd.read_csv('vuelosDeparted.csv', sep=',',low_memory=False)

In [3]:
# Load of geobase
from GeoBases import GeoBase
geo_c = GeoBase(data='ori_por', verbose=False)

In [4]:
stationstxt = ""
with open("stations.txt") as input:
    stationstxt = input.read()

In [11]:
#Extract the data from file
stations2 = stationstxt.split("\n")
#Remove last line
stations2 = stations2[:-1]
stations2 = map(lambda line: [line[0:11],float(line[13:20]),float(line[22:30]),line[41:71]], stations2)

In [6]:
weatherdf = pd.read_csv("2016.csv",header=None)
weatherdf.columns = ["id","date","type","Value1","Value2","Value3","Value4","Value5"]

In [7]:
#Get total number of days
days = len(weatherdf.groupby("date").groups.keys())

In [8]:
#Check all the types of measures that are done
print weatherdf.groupby("type").groups.keys()

['SN55', 'SN56', 'TMIN', 'DATX', 'SN52', 'SN53', 'SN31', 'MDPR', 'MNPN', 'WDFG', 'TSUN', 'SN35', 'DATN', 'SNWD', 'PGTM', 'SN51', 'AWDR', 'SX32', 'MDTX', 'SX56', 'DAPR', 'SX52', 'SX53', 'WSFI', 'SX51', 'WT05', 'WT04', 'WSF2', 'WT06', 'WT01', 'MDTN', 'PSUN', 'WT02', 'SX35', 'SX36', 'SX31', 'TOBS', 'SX33', 'EVAP', 'WT09', 'MXPN', 'WSFG', 'THIC', 'DWPR', 'AWND', 'WDF2', 'SN33', 'WT07', 'DASF', 'TAVG', 'PRCP', 'SX55', 'WT08', 'TMAX', 'MDSF', 'SNOW', 'WSF5', 'SN36', 'WT10', 'WT11', 'SN32', 'WT17', 'WT03', 'WDMV', 'WESD', 'WT18', 'WESF', 'WDF5']


In [17]:
stationsdf = pd.DataFrame(stations2)
#stationsdf.columns = ["id","lat","lng","name","asciicity"]
stationsdf.columns = ["id","lat","lng","name"]
lats = stationsdf[['lat','lng']]

In [23]:
#funciones para calcular la distancia entre los aeropuertos y estacion metereologica mas cercana
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))



In [102]:
#limitamos los datos para trabajar con más agilidad, quitar esto luego.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#vuelos = vuelos.sample(5)
#vuelos['actual_time_of_departure']

19978     2016-04-12 05:11:00
96923     2016-07-18 21:01:00
188821    2016-10-10 15:00:00
201568    2016-09-21 17:06:00
21152     2017-03-22 13:34:00
Name: actual_time_of_departure, dtype: object

In [112]:
#para que la fecha no de problemas al comparar (quizas no este bien del todo)
def tratarFecha(fecha):
    fecha = str(fecha).split(' ')[0]
    fecha = fecha.replace('-','')
    fecha = int(fecha)
    return fecha

[33, 9]

In [133]:
def tratar(lat, lon, fecha):
    #primero tratamos la fecha
    fecha = tratarFecha(fecha)
    coor = [lat,lon]
    a= closest(np.asarray(lats), coor)
    aux = stationsdf[stationsdf['lat']==a[0]]
    aux = stationsdf[stationsdf['lng']==a[1]]
    stationid = aux['id']
    s =stationid.to_string()
    stationid = str(s)
    stationid = stationid.split()[1]
    
    datatavg = weatherdf[(weatherdf["id"]==stationid)&(weatherdf['date']==fecha)].sort_values("date")
    datatavg["Value1"]=datatavg["Value1"]/10
    datatavg = datatavg.reset_index()
    TAVG = datatavg[datatavg['type']=='TAVG']['Value1']
    
    return TAVG

In [147]:
#insercion de datos de la temperatura
#vuelos["tempOrigen"] = vuelos.apply(lambda vuelos: tratarFecha(vuelos['actual_time_of_departure']), axis = 1)
vuelos["tempOrigen"] = vuelos.apply(lambda vuelos: tratar(vuelos['board_lat'],vuelos['board_lon'],vuelos['actual_time_of_departure']), axis = 1)

KeyboardInterrupt: 

In [140]:
print vuelos['actual_time_of_arrival']
print vuelos['board_lat']
print vuelos['board_lon']

19978     2016-04-12 06:45:00
96923     2016-07-18 21:51:00
188821    2016-10-10 18:19:00
201568    2016-09-22 07:22:00
21152     2017-03-22 15:11:00
Name: actual_time_of_arrival, dtype: object
19978     43.178675
96923     45.800000
188821    42.338611
201568    42.338611
21152     41.147392
Name: board_lat, dtype: float64
19978     -0.006439
96923     24.150000
188821     1.409167
201568     1.409167
21152      1.167172
Name: board_lon, dtype: float64


In [141]:
#en este punto debemos pasarle la lat y lon del vuelo

a= closest(np.asarray(lats), [45.800000, 24.150000])
aux = stationsdf[stationsdf['lat']==a[0]]
aux = stationsdf[stationsdf['lng']==a[1]]
stationid = aux['id']
s =stationid.to_string()
stationid = str(s)
stationid = stationid.split()[1]
fecha = tratarFecha('2016-07-18 21:51:00')

In [144]:
stationid

'FIE00145192'

In [145]:
#Now we extract the data from the climate table
datatavg = weatherdf[(weatherdf["id"]=='FIE00145192')].sort_values("date")
datatavg["Value1"]=datatavg["Value1"]/10
datatavg = datatavg.reset_index()
datatavg

,index,id,date,type,Value1,Value2,Value3,Value4,Value5
